# Projeto - Como pegar dados de um site com Python? - Pegando dados de ETFs do mundo inteiro.
## Desafio:
- Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e, consequentemente, do mundo. Rentabilidade, patrimônio, gestora, taxa....

## Passo a Passo
**Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python;

**Passo 2** - Importar os módulos e bibliotecas

**Passo 3** - Entender como funcionam requisições na internet

**Passo 4** - Conhecer e mapear o processo de coleta de dados no site do ETF.com.

**Passo 5** - Achar todos os elementos necessários dentro do HTML do site.

**Passo 6** - Ler a tabela de dados.

**Passo 7** - Construir a tabela final.

In [ ]:
# !pip install selenium
# !pip install html5lib
# !pip install webdriver-manager

In [1]:
#importando os pacotes necessários
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd


In [2]:
#criando o bot sendo o ChromeDriverManager()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
url = "https://www.etf.com/etfanalytics/etf-finder"
#dar um get na url, para acessar a página (get == acessar)
driver.get(url) 

## Processo de dados
- Abrir o site

- Mudar a visualização para 100

- Ler a tabela

- Avançar todas as páginas

- Trocar pra outra categoria

- Ler todas as tabelas dessa outra categoria

In [3]:
#colocar um time de 5 segs porque o python é mais rapido que o js
time.sleep(5)

#procurasse o html/java do botão que desejamos que o bot encontre dentro do site
#para encontrar o xpath do botao utilizar o navegador do bot
#pode-se utilizar um .click() para que o bot clique no botão
botao_100 =  driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]
                    /section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span
                    ''')
#executar click em cima do botão coletado anteriormente
driver.execute_script("arguments[0].click();", botao_100)

In [4]:
#web_scraping - .get aonde gostaria de entrar e utilizar o driver.find_element()
#para encontrar os "xpath" que gostaria de encontrar dentro da página
#dessa forma capturamos com xpath o número total de páginas de tabelas para automatizar o loop
numero_paginas  = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]
                                      /section[2]/div[2]/section[2]/div[2]/div/label[2]
                                      ''')
#print(numero_paginas.text)
#Agora para trocar o "of " que tem no texto por nada e deixar só o número
numero_paginas = numero_paginas.text.replace("of ","")
numero_paginas = int(numero_paginas)
print(numero_paginas)

32


In [5]:
#porém for ignora o último número-

lista_de_tabelas_por_pagina = []

for pagina in range(0, numero_paginas):
    

    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]
                             /section[2]/div[2]/div/table
                             ''')
    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabelas_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)

#concat é um recurso do pandas que transforma uma lista de tabelas com mesmo nome de coluna em uma só
base_de_dados_completa = pd.concat(lista_de_tabelas_por_pagina)
display(base_de_dados_completa)


,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B
...,...,...,...,...,...,...
8,CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--
9,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
10,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
11,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,--,--
